# Skill extraction

In [5]:
import sys
import os
import logging

# Get the project root directory (e.g., linkedin_jobs_analysis/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

from src.utils.logger import setup_logging
from src.analysis.extracting_skills_list import SkillExtractor

setup_logging()

In [2]:
STANDARD_SKILL_MAP = {
    'Solução de problemas': [
        r'\b(solu[cç][ãa]o de problemas|problem solving)\b',
        r'\b(resolu[cç][ãa]o|solu[cç][ãa]o)\b.*\b(problema|problem)\b'
    ],
    'Pensamento crítico': [
        r'\b(critical thinking|pensamento cr[ií]tico)\b'
    ],
    'Inteligência Artificial': [
        r'\b(ia|ai|intelig[êe]ncia artificial|artificial intelligence)\b',
        r'(?!.*\b(?:air|aide|caixa|main)\b)\b(ia|ai)\b',  # Avoid false matches
        r'\b(?:chatbot|llm|generative ai)\b'
    ],
    'Machine Learning': [
        r'\b(machine learning|aprendizado de m[aá]quina|ml\b)\b',
        r'\b(?:modelos?|algoritmos?)\b.*\b(?:aprendizado|machine learning)\b'
    ],
    'Cloud': [
        r'\b(cloud|nuvem|computa[cç][ãa]o em nuvem|cloud computing)\b',
        r'\b(?:gcp|azure|aws)\b.*\b(?:cloud|nuvem)\b'
    ],
    'NLP': [
        r'\b(natural language processing|processamento de linguagem natural|pln|nlp)\b',
        r'\b(?:texto|linguagem)\b.*\b(?:processamento|analysis)\b'
    ],
    'Visão computacional': [
        r'\b(vis[aã]o computacional|computer vision|cv\b)\b'
    ],
    'Feature engineering': [
        r'\bfeature engineering\b',
        r'\b(engenharia|constru[cç][ãa]o)\b.*\b(features?|caracter[ií]sticas?)\b'
    ],
    'Visualização de dados': [
        r'\b(data visualization|visualiza[cç][ãa]o de dados)\b',
        r'\b(tableau|power ?bi|looker|metabase)\b'
    ],
    'Programação': [
        r'\b(programa[cç][ãa]o|coding|desenvolvimento)\b(?!.*\b(?:sql|etl)\b)'
    ],
    'GCP': [
        r'\bgcp\b',
        r'\bgoogle cloud\b'
    ],
    'Excel': [
        r'\bexcel\b',
        r'\bmicrosoft excel\b'
    ],
    'Tableau': [
        r'\btableau\b'
    ],
    'Adaptabilidade': [
        r'\badaptabilidade\b',
        r'\b(flexibilidade|adaptability)\b'
    ],
    'Azure': [
        r'\bazure\b',
        r'\bmicrosoft azure\b'
    ],
    'Estatística': [
        r'\b(estat[ií]stica|statistics)\b',
        r'\b(an[aá]lise|analysis)\b.*\b(estat[ií]stica|statistical)\b'
    ],
    'R': [
        r'\br\b(?!.*\b(?:python|sql)\b)'  # Avoid matching parts of other words
    ],
    'Trabalho em equipe': [
        r'\b(trabalho em equipe|teamwork)\b'
    ],
    'Limpeza de dados': [
        r'\b(limpeza de dados|data cleaning)\b',
        r'\b(cleaning|limpeza)\b.*\b(data|dados)\b'
    ],
    'Arquitetura de dados': [
        r'\b(arquitetura de dados|data architecture)\b'
    ],
    'Séries temporais': [
        r'\b(s[ée]ries? temporais|time series)\b'
    ],
    'BI': [
        r'\b(business intelligence|intelig[êe]ncia de neg[óo]cios|bi\b)\b',
        r'\b(?:dashboard|relat[óo]rios?)\b.*\b(?:bi|business intelligence)\b'
    ],
    'Python': [
        r'\bpython\b(?!.*\b(?:spark|sql)\b)'
    ],
    'Teste A/B': [
        r'\ba\/b\b',
        r'\b(?:teste|test)\b.*\b(?:a\/b|ab)\b'
    ],
    'ETL': [
        r'\betl\b',
        r'\b(extract|transform|load)\b'
    ],
    'Aprendizado por reforço': [
        r'\b(reinforcement learning|aprendizado por refor[cç]o)\b'
    ],
    'PostgreSQL': [
        r'\bpostgre?s?ql\b'
    ],
    'Redes neurais': [
        r'\b(redes neurais|neural networks)\b'
    ],
    'NoSQL': [
        r'\bnosql\b'
    ],
    'Airflow': [
        r'\bairflow\b'
    ],
    'Comunicação': [
        r'\bcomunica[cç][ãa]o\b',
        r'\b(communication|comunica[cç][ãa]o)\b.*\b(?:t[ée]cnica|skills)\b'
    ],
    'Tensorflow': [
        r'\btensorflow\b',
        r'\btf\b'
    ],
    'Data Warehouse': [
        r'\b(data warehouse|datalake|data lake)\b'
    ],
    'Kubernetes': [
        r'\bkubernetes\b',
        r'\bk8s\b'
    ],
    'Pipeline': [
        r'\b(pipeline|data pipeline)\b'
    ],
    'Governança de dados': [
        r'\b(governan[cç]a de dados|data governance)\b'
    ],
    'Hadoop': [
        r'\bhadoop\b'
    ],
    'Manipulação de dados': [
        r'\b(data wrangling|manipula[cç][ãa]o de dados)\b'
    ],
    'Qualidade dos dados': [
        r'\b(data quality|qualidade dos dados)\b'
    ],
    'PyTorch': [
        r'\bpytorch\b'
    ],
    'MySQL': [
        r'\bmysql\b'
    ],
    'Análise de negócios': [
        r'\b(business analytics|an[aá]lise de neg[óo]cios)\b'
    ],
    'Deep Learning': [
        r'\b(deep learning|aprendizado profundo)\b'
    ],
    'Big Data': [
        r'\bbig ?data\b'
    ],
    'Data storytelling': [
        r'\b(data storytelling|storytelling)\b'
    ],
    'Mineração de dados': [
        r'\b(minera[cç][ãa]o de dados|data mining)\b'
    ],
    'SQL': [
        r'\bsql\b(?!.*\b(?:server|mysql)\b)'
    ],
    'Power BI': [
        r'\bpower ?bi\b'
    ],
    'Spark': [
        r'\bspark\b',
        r'\bpyspark\b'
    ],
    'API': [
        r'\bapi\b(?!.*\b(?:google|azure)\b)'
    ],
    'Scikit-learn': [
        r'\bscikit[-\s]?learn\b'
    ],
    'Pandas': [
        r'\bpandas\b(?!.*\b(?:aws|animal)\b)'
    ],
    'Docker': [
        r'\bdocker\b'
    ],
    'AWS': [
        r'\baws\b',
        r'\bamazon web services\b'
    ],
    'MATLAB': [
        r'\bmatlab\b'
    ],
    'SQL Server': [
        r'\bsql server\b'
    ],
    'NumPy': [
        r'\bnumpy\b'
    ]
}

In [3]:
def test_skill_extractor(test_cases = [
        (
            "Precisamos de um profissional com experiência em Python e SQL.",
            ['Python', 'SQL']
        ),
        (
            "Conhecimento em Machine Learning (ML) e PowerBI é essencial.",
            ['Machine Learning', 'Power BI']
        ),
        (
            "Domínio de AWS, Azure e GCP para cloud computing.",
            ['Cloud']
        ),
        (
            "Experiência com PySpark e Spark para processamento de dados.",
            ['spark', 'pyspark']  # Not in standard map
        ),
        (
            "Habilidade em Excel avançado e análise de dados.",
            ['excel']
        ),
        (
            "Conhecimento em Airflow para orquestração de pipelines.",
            ['airflow']
        ),
        (
            "Não há requisitos técnicos específicos para esta vaga.",
            []
        ),
        (
            "Necessário conhecimento em Python e pandas para análise de dados.",
            ['Python', 'pandas']
        ),
        (
            "A palavra 'excelente' não deve ser confundida com Excel.",
            []
        ),
        (
            "MLOps e Machine Learning são importantes para a vaga.",
            ['Machine Learning']  # Assuming MLOps isn't in our test skills
        )
    ]):
    """Test function for SkillExtractor with various edge cases"""
    # Sample skill list and mapping
    test_skills = [
        "python", "sql", "machine learning", "pandas", 
        "power bi", "tableau", "aws", "azure", "gcp",
        "spark", "pyspark", "airflow", "excel"
    ]
    
    test_skill_map = {
        'Python': ['python', 'python3'],
        'SQL': ['sql', 'structured query language'],
        'Machine Learning': ['machine learning', 'ml', 'aprendizado de máquina'],
        'Power BI': ['power bi', 'powerbi'],
        'Cloud': ['aws', 'azure', 'gcp', 'cloud computing']
    }

    # Initialize extractor
    extractor = SkillExtractor(skill_list=test_skills, standard_skill_map=test_skill_map)


    failed = 0
    for i, (text, expected) in enumerate(test_cases, 1):
        try:
            result = extractor.extract_skills(text)
            # Convert both to sets for unordered comparison
            result_set = set(result)
            expected_set = set(expected)
            
            if result_set != expected_set:
                print(f"\nFAIL Case {i}: '{text}'")
                print(f"  Expected: {sorted(expected)}")
                print(f"  Got:      {sorted(result)}")
                print(f"  Missing:  {sorted(expected_set - result_set)}")
                print(f"  Extra:    {sorted(result_set - expected_set)}")
                failed += 1
            else:
                print(f"PASS Case {i}: '{text}' => {result}")
        except Exception as e:
            print(f"ERROR Case {i}: Failed to process - {str(e)}")
            failed += 1

    print(f"\nTest Results: {len(test_cases)-failed} passed, {failed} failed")
    return failed == 0

In [6]:
test_skill_extractor()

2025-05-18 13:44:28,783 - src.analysis.extracting_skills_list - SUCCESS - Successfully loaded spaCy model: pt_core_news_lg
2025-05-18 13:44:28,785 - src.analysis.extracting_skills_list - INFO - Prepared matcher with 13 skill patterns.


PASS Case 1: 'Precisamos de um profissional com experiência em Python e SQL.' => ['Python', 'SQL']

FAIL Case 2: 'Conhecimento em Machine Learning (ML) e PowerBI é essencial.'
  Expected: ['Machine Learning', 'Power BI']
  Got:      ['Machine Learning']
  Missing:  ['Power BI']
  Extra:    []
PASS Case 3: 'Domínio de AWS, Azure e GCP para cloud computing.' => ['Cloud']
PASS Case 4: 'Experiência com PySpark e Spark para processamento de dados.' => ['spark', 'pyspark']
PASS Case 5: 'Habilidade em Excel avançado e análise de dados.' => ['excel']
PASS Case 6: 'Conhecimento em Airflow para orquestração de pipelines.' => ['airflow']
PASS Case 7: 'Não há requisitos técnicos específicos para esta vaga.' => []
PASS Case 8: 'Necessário conhecimento em Python e pandas para análise de dados.' => ['pandas', 'Python']

FAIL Case 9: 'A palavra 'excelente' não deve ser confundida com Excel.'
  Expected: []
  Got:      ['excel']
  Missing:  []
  Extra:    ['excel']
PASS Case 10: 'MLOps e Machine Learni

False

---

In [1]:
import re
from typing import Dict, List
import pandas as pd


In [2]:
import sys
import os

# Get the project root directory (e.g., linkedin_jobs_analysis/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

# Now import from src
from config.analysis import ROLE_PATTERNS, SPECIAL_CASES
from src.analysis.analysis_utils import test_classifier, title_classifier


In [3]:
def title_classifier(title: str) -> str:
    """
    Classifies job titles into standardized categories using pattern matching.
    
    Args:
        title: The raw job title to classify
        
    Returns:
        str: The standardized job category or 'Outros' if no match found
    """
    if not isinstance(title, str) or len(title.strip()) == 0:
        return 'Outros'
    
    title_lower = title.lower().strip()
    matches = []

    for role, patterns in ROLE_PATTERNS.items():
        for pattern in patterns:
            if re.search(pattern, title_lower):
                matches.append(role)
                break

    if matches:
        for candidate_role in matches:
            special_rules = SPECIAL_CASES.get(candidate_role, [])
            for exclude_pattern, new_role in special_rules:
                if re.search(exclude_pattern, title_lower):
                    if new_role:
                        return new_role
                    matches.remove(candidate_role)

        for role in ROLE_PATTERNS.keys():
            if role in matches:
                return role

    if any(word in title_lower for word in ['dados', 'data']):
        return 'Outros Dados'

    return 'Outros'

In [4]:
title_classifier('Analista de dados e automação com IA - Pleno	')

'Analista de Dados'

In [5]:
jobs_classified = pd.read_csv('jobs_classified.csv')
jobs_classified_tuple = list(zip(jobs_classified['job_title'], jobs_classified['classified_job_title']))

test_classifier(jobs_classified_tuple)

FAIL: 'Analista de dados e automação com IA - Pleno'
  Expected: Engenheiro de IA
  Got:      Analista de Dados
FAIL: 'Engenheiro (a) de IA'
  Expected: Outros
  Got:      Engenheiro de IA
FAIL: 'Engenheiro (a) de Automação Sr / CS / Uberlândia - MG'
  Expected: Engenheiro de IA
  Got:      Outros
FAIL: 'Engenheiro (a) de IA'
  Expected: Outros
  Got:      Engenheiro de IA
FAIL: 'Engenheiro (a) de IA'
  Expected: Outros
  Got:      Engenheiro de IA
FAIL: 'Engenheiro (a) de IA'
  Expected: Outros
  Got:      Engenheiro de IA
FAIL: 'Engenheiro(a) de Software (Fullstack) | IT Energy'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro(a) de Software Pleno – Plataforma de IA'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro de Automação Industrial'
  Expected: Engenheiro de IA
  Got:      Outros
FAIL: 'Engenheiro(a) de Software Frontend'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro(a) de Software (.NET) Jr/Pleno | Electr

In [27]:
ROLE_PATTERNS['Cientista de Dados']

['\\b(data scientist|cientista de dados)\\b',
 '\\b(ml|machine learning|deep learning|llm|nlp)\\b',
 '(pesquisador|research).*(dados|data)',
 'ci[êe]ncia de dados',
 '\\bgenai\\b',
 '(computer vision|visão computacional)']

In [7]:
jobs_classified[jobs_classified['classified_job_title']=='Engenheiro de Dados'].sample(20)

,Unnamed: 0,job_title,classified_job_title
2752,2752,Azure Data Engineer,Engenheiro de Dados
3059,3059,Blockchain Data Engineer (Senior/Lead) ID34521,Engenheiro de Dados
3035,3035,Data Engineer with PowerCenter Experience - Re...,Engenheiro de Dados
1076,1076,Engenheiro(a) de Dados (DevOps/DataOps),Engenheiro de Dados
737,737,Engenheiro de Dados - Trabalho Remoto | REF#25...,Engenheiro de Dados
1886,1886,Engenheiro/ Engenheira de Dados Pyspark,Engenheiro de Dados
833,833,Engenheiro(a) de Dados,Engenheiro de Dados
1911,1911,Engenheiro de dados,Engenheiro de Dados
2398,2398,Data Engineer,Engenheiro de Dados
2893,2893,Banco de Talentos Data Engineer,Engenheiro de Dados


---

# Location

In [8]:
import pandas as pd
import re


In [ ]:
def standardize_locations(df: pd.DataFrame, location_col: str = 'location') -> pd.DataFrame:
    """
    Standardizes location data into separate city, state, and country columns.
    Handles cases like "São Paulo, Brasil" correctly.
    """
    
    df['city'] = None
    df['state'] = None
    df['country'] = 'Brasil'
    
    state_mapping = {
        'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas',
        'BA': 'Bahia', 'CE': 'Ceará', 'DF': 'Distrito Federal',
        'ES': 'Espírito Santo', 'GO': 'Goiás', 'MA': 'Maranhão',
        'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul',
        'MG': 'Minas Gerais', 'PA': 'Pará', 'PB': 'Paraíba',
        'PR': 'Paraná', 'PE': 'Pernambuco', 'PI': 'Piauí',
        'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte',
        'RS': 'Rio Grande do Sul', 'RO': 'Rondônia',
        'RR': 'Roraima', 'SC': 'Santa Catarina',
        'SP': 'São Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'
    }
    
    state_mapping = {**{v: k for k, v in state_mapping.items()}, **state_mapping}
    
    def extract_location(location):
        if pd.isna(location):
            return (None, None, None)
            
        location = str(location).strip()
        
        if re.search(r',\s*Brasil$', location, flags=re.IGNORECASE):
            city = re.sub(r',\s*Brasil$', '', location, flags=re.IGNORECASE).strip()
            state = None
            for state_abbr, state_name in state_mapping.items():
                if city.lower() == state_name.lower():
                    state = state_abbr
                    break
            return (city.title(), state, 'Brasil')
        
        match = re.match(r'^(?P<city>[^,]+),\s*(?P<state>[A-Z]{2})$', location)
        if match:
            return (match.group('city').title(), match.group('state').upper(), 'Brasil')
        
        match = re.match(r'^(?P<city>[^,]+),\s*(?P<state>.+)$', location)
        if match:
            state = match.group('state')
            if state.lower() in ['brasil', 'brazil']:
                return (match.group('city').title(), None, 'Brasil')
            state_abbr = state_mapping.get(state.title(), state)
            return (match.group('city').title(), state_abbr, 'Brasil')
        
        match = re.match(r'^(?P<city>.+)\s+e\s+Região$', location)
        if match:
            city = match.group('city')
            state = state_mapping.get(city.title(), None)
            return (city.title(), state, 'Brasil')
        
        return (location.title(), state_mapping.get(location.title(), None), 'Brasil')
    
    df[['city', 'state', 'country']] = df[location_col].apply(
        lambda x: pd.Series(extract_location(x))
    )
    
    return df

In [46]:
job_copy

,job_id,work_model,keyword,scrape_date,job_title,company_name,location,num_applicants,xp_level,job_type,job_sectors,job_description,classified_job_title,post_date,city,state,country
0,4219203458,Presencial,Analista de Dados,2025-05-17,Analista de Dados e Serviço ao Cliente Junior,Cielo,"Barueri, SP",177.0,Assistente,Tempo integral,Atividades de serviços financeiros,"Job Description\nSomos mais que uma máquina, s...",Analista de Dados,03-05-2025,Barueri,SP,Brasil
1,4229491792,Presencial,Analista de Dados,2025-05-17,Analista de dados de negócios,SulAmérica,"São Paulo, SP",117.0,Pleno-sênior,Tempo integral,Seguros e previdência complementar e Serviços ...,A SulAmérica há mais de 125 anos se dedica a e...,Analista de Dados,16-05-2025,São Paulo,SP,Brasil
2,4209520797,Presencial,Analista de Dados,2025-05-17,Analista de Dados Júnior,Banco Fibra,"São Paulo, SP",163.0,Não aplicável,Tempo integral,Bancos,Somos um Banco que trabalha na busca do melhor...,Analista de Dados,19-04-2025,São Paulo,SP,Brasil
3,4201072662,Presencial,Analista de Dados,2025-05-17,"Analista de Dados - Júnior, Pleno e Sênior | R...",Capco,"São Paulo, SP",112.0,Pleno-sênior,Tempo integral,Atividades de serviços financeiros,SOBRE A CAPCO\nA Capco é uma consultoria globa...,Analista de Dados,26-04-2025,São Paulo,SP,Brasil
4,4223039814,Presencial,Analista de Dados,2025-05-17,Analista de Dados & Analytics Pleno | Riscos e...,C6 Bank,"São Paulo, SP",89.0,Assistente,Tempo integral,Bancos,Nossa área de Processos & Controles\nA área de...,Analista de Dados,10-05-2025,São Paulo,SP,Brasil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3500,4231053333,Presencial,Analista de BI,2025-05-18,Programador de Sistemas de Informação,Sebratel,"Porto Alegre, RS",NaN,Assistente,Tempo integral,Telecomunicações,"DESCRIÇÃO\nA SEBRATEL, empresa referência no s...",Outros,16-05-2025,Porto Alegre,RS,Brasil
3501,4232436842,Híbrido,Analista de BI,2025-05-18,Analista de BI (Inteligência de Mercado) - MG,Drogaria Araujo S/A,"Belo Horizonte, MG",NaN,Pleno-sênior,Tempo integral,Comércio varejista,"A Drogaria Araujo, a maior Rede de Varejo Farm...",Analista de BI,NaN,Belo Horizonte,MG,Brasil
3502,4232150473,Híbrido,Analista de BI,2025-05-18,Analista de bi business intelligence,Netvagas,"São Paulo, SP",NaN,Pleno-sênior,Tempo integral,Fornecimento e gestão de recursos humanos,"Na Connect For People, acreditamos em conectar...",Analista de BI,17-05-2025,São Paulo,SP,Brasil
3503,4232380566,Remoto,Analista de BI,2025-05-18,Analista de bi pleno,Netvagas,Brasil,NaN,Assistente,Tempo integral,Fornecimento e gestão de recursos humanos,"A FIAP é uma faculdade de tecnologia, inovação...",Analista de BI,17-05-2025,None,None,Brasil


In [43]:
jobs = pd.read_csv('../data/processed/df_jobs_classified.csv')
job_copy = jobs.copy()
df = standardize_locations(jobs, location_col='location')

In [40]:
df[df['location'].str.contains('remoto')]

,job_id,work_model,keyword,scrape_date,job_title,company_name,location,num_applicants,xp_level,job_type,job_sectors,job_description,classified_job_title,post_date,city,state,country


In [50]:
job_copy['city'].unique()

array(['Barueri', 'São Paulo', 'Maracanaú', 'Brasília', 'Curitiba',
       'Guará', 'Blumenau', 'Campinas', 'Fortaleza', 'Indaial',
       'Rio de Janeiro', 'Contagem', 'Caxias do Sul', 'Mogi das Cruzes',
       'Uberlândia', 'Porto Alegre', 'Cravinhos', 'Pereiro', 'Itajaí',
       'Betim', 'Macaé', 'Goiânia', 'Palotina', 'Campo Grande', 'Serra',
       'Tijucas', 'Novo Hamburgo', 'Osasco', 'Maringá', 'Cachoeirinha',
       'Vitória', 'Mossoró', 'Sorocaba', 'Dois Irmãos', 'Arcos',
       'Jaraguá do Sul', 'Joinville', 'Capivari', 'Várzea Grande', None,
       'Marília', 'Aracaju', 'Pato Branco', 'Franca', 'Rio do Sul',
       'Sumaré', 'Ibirama', 'Belo Horizonte', 'Recife', 'Manaus',
       'Embu das Artes', 'Eldorado do Sul', 'Viçosa',
       'São José do Rio Preto', 'Leopoldina', 'Monte Belo', 'Londrina',
       'Salvador', 'Morrinhos', 'Vila Velha', 'Ribeirão Preto', 'Cuiabá',
       'Florianópolis', 'Belém', 'Jundiaí', 'Santo André', 'Santa Rosa',
       'Guarulhos', 'Matão', 'Luca

In [53]:
df['city'].unique().sort()
df['city'].unique()

array(['Barueri', 'São Paulo', 'Maracanaú', 'Brasília', 'Curitiba',
       'Guará', 'Blumenau', 'Campinas', 'Fortaleza', 'Indaial',
       'Rio De Janeiro', 'Contagem', 'Caxias Do Sul', 'Mogi Das Cruzes',
       'Uberlândia', 'Porto Alegre', 'Cravinhos', 'Pereiro', 'Itajaí',
       'Betim', 'Macaé', 'Goiânia', 'Palotina', 'Campo Grande', 'Serra',
       'Tijucas', 'Novo Hamburgo', 'Osasco', 'Maringá', 'Cachoeirinha',
       'Vitória', 'Mossoró', 'Sorocaba', 'Dois Irmãos', 'Arcos',
       'Jaraguá Do Sul', 'Joinville', 'Capivari', 'Várzea Grande',
       'Brasil', 'Marília', 'Aracaju', 'Pato Branco', 'Franca',
       'Rio Do Sul', 'Sumaré', 'Ibirama', 'Belo Horizonte', 'Recife',
       'Manaus', 'Embu Das Artes', 'Eldorado Do Sul', 'Viçosa',
       'São José Do Rio Preto', 'Leopoldina', 'Monte Belo', 'Londrina',
       'Salvador', 'Morrinhos', 'Vila Velha', 'Ribeirão Preto', 'Cuiabá',
       'Florianópolis', 'Belém', 'Jundiaí', 'Santo André', 'Santa Rosa',
       'Guarulhos', 'Matão', '

In [72]:
job_copy[(job_copy[['city', 'state', 'country']] != df[['city', 'state', 'country']]).any(axis=1)][['city', 'state', 'country']]

,city,state,country
23,São Paulo,None,Brasil
25,São Paulo,None,Brasil
26,São Paulo,None,Brasil
30,Rio de Janeiro,RJ,Brasil
31,São Paulo,None,Brasil
...,...,...,...
3485,None,None,Brasil
3486,None,None,Brasil
3490,Rio de Janeiro,None,Brasil
3503,None,None,Brasil


In [89]:
mask = (job_copy[['city', 'state', 'country']] != df[['city', 'state', 'country']])
# Show both DataFrames side by side where they differ
differences = pd.concat([job_copy[mask.any(axis=1)], df[mask.any(axis=1)]], axis=1, keys=['job_copy', 'df'])
differences[[('job_copy', 'city'), ('job_copy', 'state'), ('job_copy', 'country'), ('df', 'city'), ('df', 'state'), ('df', 'country')]].sample(10)

job_copy                                  df              
                      city state country                  city state country
830              São Paulo  None  Brasil             São Paulo    SP  Brasil
724                   None  None  Brasil                Brasil  None  Brasil
3351        Rio de Janeiro  None  Brasil        Rio De Janeiro  None  Brasil
2419  São José dos Pinhais    PR  Brasil  São José Dos Pinhais    PR  Brasil
3399        Rio de Janeiro  None  Brasil        Rio De Janeiro  None  Brasil
2345             São Paulo  None  Brasil             São Paulo    SP  Brasil
2027        Rio de Janeiro    RJ  Brasil        Rio De Janeiro    RJ  Brasil
1573                  None  None  Brasil                Brasil  None  Brasil
1997         Caxias do Sul    RS  Brasil         Caxias Do Sul    RS  Brasil
496                   None  None  Brasil                Brasil  None  Brasil

In [56]:
df

,job_id,work_model,keyword,scrape_date,job_title,company_name,location,num_applicants,xp_level,job_type,job_sectors,job_description,classified_job_title,post_date,city,state,country
0,4219203458,Presencial,Analista de Dados,2025-05-17,Analista de Dados e Serviço ao Cliente Junior,Cielo,"Barueri, SP",177.0,Assistente,Tempo integral,Atividades de serviços financeiros,"Job Description\nSomos mais que uma máquina, s...",Analista de Dados,03-05-2025,Barueri,SP,Brasil
1,4229491792,Presencial,Analista de Dados,2025-05-17,Analista de dados de negócios,SulAmérica,"São Paulo, SP",117.0,Pleno-sênior,Tempo integral,Seguros e previdência complementar e Serviços ...,A SulAmérica há mais de 125 anos se dedica a e...,Analista de Dados,16-05-2025,São Paulo,SP,Brasil
2,4209520797,Presencial,Analista de Dados,2025-05-17,Analista de Dados Júnior,Banco Fibra,"São Paulo, SP",163.0,Não aplicável,Tempo integral,Bancos,Somos um Banco que trabalha na busca do melhor...,Analista de Dados,19-04-2025,São Paulo,SP,Brasil
3,4201072662,Presencial,Analista de Dados,2025-05-17,"Analista de Dados - Júnior, Pleno e Sênior | R...",Capco,"São Paulo, SP",112.0,Pleno-sênior,Tempo integral,Atividades de serviços financeiros,SOBRE A CAPCO\nA Capco é uma consultoria globa...,Analista de Dados,26-04-2025,São Paulo,SP,Brasil
4,4223039814,Presencial,Analista de Dados,2025-05-17,Analista de Dados & Analytics Pleno | Riscos e...,C6 Bank,"São Paulo, SP",89.0,Assistente,Tempo integral,Bancos,Nossa área de Processos & Controles\nA área de...,Analista de Dados,10-05-2025,São Paulo,SP,Brasil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3500,4231053333,Presencial,Analista de BI,2025-05-18,Programador de Sistemas de Informação,Sebratel,"Porto Alegre, RS",NaN,Assistente,Tempo integral,Telecomunicações,"DESCRIÇÃO\nA SEBRATEL, empresa referência no s...",Outros,16-05-2025,Porto Alegre,RS,Brasil
3501,4232436842,Híbrido,Analista de BI,2025-05-18,Analista de BI (Inteligência de Mercado) - MG,Drogaria Araujo S/A,"Belo Horizonte, MG",NaN,Pleno-sênior,Tempo integral,Comércio varejista,"A Drogaria Araujo, a maior Rede de Varejo Farm...",Analista de BI,NaN,Belo Horizonte,MG,Brasil
3502,4232150473,Híbrido,Analista de BI,2025-05-18,Analista de bi business intelligence,Netvagas,"São Paulo, SP",NaN,Pleno-sênior,Tempo integral,Fornecimento e gestão de recursos humanos,"Na Connect For People, acreditamos em conectar...",Analista de BI,17-05-2025,São Paulo,SP,Brasil
3503,4232380566,Remoto,Analista de BI,2025-05-18,Analista de bi pleno,Netvagas,Brasil,NaN,Assistente,Tempo integral,Fornecimento e gestão de recursos humanos,"A FIAP é uma faculdade de tecnologia, inovação...",Analista de BI,17-05-2025,Brasil,None,Brasil
